In [ ]:
import os
import rioxarray 
import requests
import zipfile
import io
import rasterio
import unicodedata
import joblib
import math

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.patheffects as pe
import geopandas as gpd
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

from rasterio.transform import xy
from rasterio.features import rasterize
from shapely.geometry import Point
from sklearn.impute import SimpleImputer
from matplotlib.colors import ListedColormap, BoundaryNorm
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datetime import datetime, timedelta

In [ ]:
N_DAYS = 5
PRED_DAYS = 10

In [ ]:
Data_folder  = '../Inputs/Feature_Maps'
#Change the Dates to change the days you want to predict
Dates  = ['20211222', '20211223', '20211224', '20211225', '20211226']
Features = ['TMP', 'TX', 'TN', 'RH', 'WSPD', 'WDIR', 'TP', 'PRES2M', 'SQRT_SEA_DEM_LAT']

In [ ]:
def read_tif_to_array(var, date):
    
    if (var == 'SQRT_SEA_DEM_LAT'):
        path = os.path.join(Data_folder, f"{var}.tif")
    else:
        path = os.path.join(Data_folder, var, f"{var}_{date}.tif")
        
    if not os.path.isfile(path):
        raise FileNotFoundError(f"Không tìm thấy file: {path}")
        
    da = rioxarray.open_rasterio(path, masked=True).squeeze('band', drop=True)
    arr = da.values         
    return arr.T          

In [ ]:
arrays = {var: {} for var in Features}

for date in Dates:
    for var in Features:
        arrays[var][date] = read_tif_to_array(var, date)

In [ ]:
lags = [5, 4, 3, 2, 1]

col_names = []
data_cols = []

for var in Features:
    for date, lag in zip(Dates, lags):
        col_names.append(f"{var}_lag_{lag}")
        data_cols.append(arrays[var][date].flatten(order='F'))

data_matrix = np.column_stack(data_cols)
df = pd.DataFrame(data_matrix, columns=col_names)

In [ ]:
shp_admin1 = "../Inputs/vn_shp/gadm41_VNM_1.shp"
gdf = gpd.read_file(shp_admin1)

provinces_north = [
    "Vĩnh Phúc", "Quảng Ninh", "Hải Phòng", "Hưng Yên", "Thái Bình",
    "Hà Nam", "Nam Định", "Ninh Bình", "Hà Giang", "Cao Bằng",
    "Bắc Kạn", "Tuyên Quang", "Lào Cai", "Yên Bái", "Thái Nguyên",
    "Lạng Sơn", "Điện Biên", "Lai Châu", "Sơn La", "Hoà Bình",
    "Bắc Giang", "Bắc Ninh", "Hải Dương", "Hà Nội", "Phú Thọ"
]

gdf_north = gdf[gdf["NAME_1"].isin(provinces_north)]

In [ ]:
tif_path = os.path.join(Data_folder, 'SQRT_SEA_DEM_LAT.tif')
shp_path = 'vn_north_shp/north_provinces.shp'

da = rioxarray.open_rasterio(tif_path, masked=True).squeeze('band', drop=True)
transform = da.rio.transform()
crs = da.rio.crs
raster_vals = da.values
xmin, xmax = da.x.values.min(), da.x.values.max()
ymin, ymax = da.y.values.min(), da.y.values.max()

gdf = gdf_north.to_crs(crs)

def normalize_name(name):
    s = ''.join(w.capitalize() for w in name.split())
    s = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in s if not unicodedata.combining(c))

gdf['code'] = gdf['NAME_1'].map(normalize_name)

province_ids = {n: i+1 for i,n in enumerate(gdf['NAME_1'])}
shapes = [(geom, province_ids[n]) for geom,n in zip(gdf.geometry, gdf['NAME_1'])]
province_arr = rasterize(
    shapes,
    out_shape=da.shape,
    transform=transform,
    fill=0,
    dtype='int32'
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8))

gdf.plot(ax=ax1, facecolor='none', edgecolor='black', linewidth=0.8)
for _, row in gdf.iterrows():
    c = row.geometry.centroid
    ax1.text(
        c.x, c.y, row['code'],
        ha='center', va='center',
        fontsize=8, color='black',
        path_effects=[pe.withStroke(linewidth=1, foreground='white')]
    )
ax1.set_title("25 tỉnh miền Bắc (shapefile)")
ax1.set_axis_off()

ax2.imshow(
    raster_vals,
    extent=[xmin, xmax, ymin, ymax],
    origin='upper',
    cmap='viridis'
)
gdf.boundary.plot(ax=ax2, edgecolor='white', linewidth=0.5)
for _, row in gdf.iterrows():
    c = row.geometry.centroid
    ax2.text(
        c.x, c.y, row['code'],
        ha='center', va='center',
        fontsize=6, color='white',
        path_effects=[pe.withStroke(linewidth=1, foreground='black')]
    )
ax2.set_title("25 tỉnh miền Bắc (SQRT_SEA_DEM_LAT tif)")
ax2.set_axis_off()

mappable = ax2.images[0]
plt.colorbar(mappable, ax=ax2, fraction=0.046, pad=0.04, label=da.name or 'Value')

plt.tight_layout()
plt.show()

In [ ]:
id_to_name = {code: name for name, code in province_ids.items()}

prov_flat = province_arr.flatten(order='F')

prov_names = [id_to_name.get(code) for code in prov_flat]

def format_name(s):
    if not isinstance(s, str):
        return None
    s = s.replace('Đ', 'D').replace('đ', 'd')
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    return ''.join(word.capitalize() for word in s.split())

prov_flat   = province_arr.ravel()
prov_names  = [id_to_name.get(code) for code in prov_flat]
formatted   = [format_name(name) for name in prov_names]
df['province'] = formatted

In [ ]:
n_rows, n_cols = province_arr.shape
mask_missing   = df['province'].isna().values
rows, cols = np.unravel_index(np.arange(n_rows*n_cols), (n_rows, n_cols), order='F')
miss_rows = rows[mask_missing]
miss_cols = cols[mask_missing]
xs, ys    = rasterio.transform.xy(transform, miss_rows, miss_cols, offset='center')
gdf_pts = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(xs, ys),
    crs=crs,
    index=np.nonzero(mask_missing)[0]
)

proj_crs = "EPSG:32648"

gdf_proj     = gdf.to_crs(proj_crs)
gdf_pts_proj = gdf_pts.to_crs(proj_crs)

joined = gpd.sjoin_nearest(
    gdf_pts_proj,
    gdf_proj[['code','geometry']],
    how='left',
    distance_col='dist'
)

df.loc[joined.index, 'province'] = joined['code'].values
df['province'] = df['province'].apply(format_name)

In [ ]:
target_code = 'BacKan'
code_to_id = {normalize_name(name): _id for name, _id in province_ids.items()}
target_id = code_to_id[target_code]

coords = np.argwhere(province_arr == target_id)
rows, cols = coords[:,0], coords[:,1]

xs, ys = xy(transform, rows, cols, offset='center')

fig, ax = plt.subplots(figsize=(8,8))
ax.imshow(raster_vals, extent=[xmin, xmax, ymin, ymax], origin='upper', cmap='viridis')
gdf.boundary.plot(ax=ax, edgecolor='white', linewidth=0.8)

ax.scatter(xs, ys, s=5, c='red', alpha=0.6, label=f'{target_code} pixels')

ax.set_title(f"Tất cả pixel thuộc tỉnh {target_code} (đỏ)")
ax.set_axis_off()
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
pop_df = pd.read_excel('../Inputs/population_data_cac_tinh_mien_bac.xlsx')

pop_df['province'] = pop_df['province'].apply(format_name)

pop_df.set_index('province', inplace=True)

df['urbanization_rate'] = df['province'].map(pop_df['urbanization_rate'])
df['population_density'] = df['province'].map(pop_df['population_density'])
df['SQRT_SEA_DEM_LAT'] = df['SQRT_SEA_DEM_LAT_lag_1']

def get_season(month):
    if month in (12, 1, 2):
        return 'Winter'
    elif month in (3, 4, 5):
        return 'Spring'
    elif month in (6, 7, 8):
        return 'Summer'
    else:
        return 'Autumn'

season = get_season(int(Dates[0][4:6]))
df['season'] = season

for lag in range(1, N_DAYS + 1):
    df[f'diffusion_conditions_lag_{lag}'] = (
        df[f'WSPD_lag_{lag}'] * df[f'TP_lag_{lag}']
    )

In [ ]:
lag_features = ['WSPD', 'WDIR', 'TMP', 'TX', 'TN', 'TP', 'RH', 'PRES2M', 'diffusion_conditions']

features = ([
    'SQRT_SEA_DEM_LAT', 'urbanization_rate', 'population_density', 'season'] +
    [f'{feature}_lag_{i}' for feature in lag_features for i in range(1, N_DAYS + 1)]
)

X = df[features].copy()

In [ ]:
categorical_columns = X.select_dtypes(include=['object']).columns

In [ ]:
label_encoder = LabelEncoder()

for col in categorical_columns:
    X[col] = label_encoder.fit_transform(X[col].astype(str))

In [ ]:
scaler = StandardScaler()
X_scaled_array = scaler.fit_transform(X)

In [ ]:
X = pd.DataFrame(
    X_scaled_array,
    index=X.index,
    columns=X.columns
)

In [ ]:
mask_missing_any = X.isna().any(axis=1)

In [ ]:
imp = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imp.fit_transform(X), columns=X.columns)

In [ ]:
models_folder = '../Models/Final_Models'

In [ ]:
def round_with_thresholds(aqi_vals, th):
    return np.digitize(aqi_vals, np.sort(th))

In [ ]:
targets = [f'AQI_cat_target_{day}' for day in range(PRED_DAYS)]

models_dict = {'XGB': {}, 'LGBM': {}, 'CatBoost': {}}

for model_name in ['XGB', 'LGBM', 'CatBoost']:
    for target in targets:
        fn = os.path.join(models_folder, f'{model_name}_{target}_model.pkl')
        try:
            models_dict[model_name][target] = joblib.load(fn)
            print(f"Loaded {fn}")
        except FileNotFoundError:
            print(f"Model file {fn} not found.")

best_weights_list = np.load(os.path.join(models_folder, 'best_weights.npy'), allow_pickle=True)
thresholds_per_day = np.load(os.path.join(models_folder, 'thresholds_per_day.npy'), allow_pickle=True)
print("Loaded best_weights_list and thresholds_per_day")

In [ ]:
bst = models_dict['XGB'][targets[0]]
feature_order = bst.feature_names  
X = X[feature_order]

In [ ]:
xgb_preds = np.column_stack([
    models_dict['XGB'][target].predict(xgb.DMatrix(X))
    for target in targets
])

lgbm_preds = np.column_stack([
    models_dict['LGBM'][target].predict(X)
    for target in targets
])

catboost_preds = np.column_stack([
    models_dict['CatBoost'][target].predict(X)
    for target in targets
])

In [ ]:
n_samples, n_days = xgb_preds.shape
rounded_preds = np.full((n_samples, PRED_DAYS), np.nan, dtype=float)

In [ ]:
for day in range(PRED_DAYS):
    w_xgb, w_lgbm, w_cat = best_weights_list[day]
    ens = (
        w_xgb  * xgb_preds[:, day] +
        w_lgbm * lgbm_preds[:, day] +
        w_cat  * catboost_preds[:, day]
    )
    rounded = round_with_thresholds(ens, thresholds_per_day[day]).astype(float)
    rounded[mask_missing_any] = np.nan
    rounded_preds[:, day] = rounded

In [ ]:
outputs_df = pd.DataFrame(rounded_preds, columns=targets)
final_df = pd.concat([df[features], outputs_df], axis=1)

In [ ]:
transform = da.rio.transform()
crs       = da.rio.crs
height, width = province_arr.shape

meta = {
    'driver': 'GTiff',
    'dtype': 'uint8',
    'nodata': 255,
    'width': width,
    'height': height,
    'count': 1,
    'crs': crs,
    'transform': transform,
    'photometric': 'palette',
    'interleave': 'band'
}

palette = {
    0: (  0, 228,   0),
    1: (255, 255,   0),
    2: (255, 126,   0),
    3: (255,   0,   0),
    4: (143,  63, 151),
    5: (126,   0,  35),
    255: (0, 0, 0)
}

last_date = datetime.strptime(Dates[-1], '%Y%m%d').date()
start_date = last_date + timedelta(days=1)
window = 10
new_dates = [
    (start_date + timedelta(days=i)).strftime('%Y%m%d')
    for i in range(window)
]
end_date = start_date + timedelta(days=window-1)
out_folder = f"../Outputs/{start_date.strftime('%Y%m%d')}_to_{end_date.strftime('%Y%m%d')}"

os.makedirs(out_folder, exist_ok=True)

for target in targets:
    arr_f = outputs_df[target].values.reshape(height, width)
    arr = np.where(np.isnan(arr_f), meta['nodata'], arr_f).astype(np.uint8)

    out_path = os.path.join(out_folder, f"{target}.tif")
    with rasterio.open(out_path, 'w', **meta) as dst:
        dst.write(arr, 1)
        dst.write_colormap(1, palette)
        mask = np.where(arr == meta['nodata'], 0, 255).astype('uint8')
        dst.write_mask(mask)

    print(f"Saved: {out_path}")

In [ ]:
shp_path = '../Inputs/vn_north_shp/north_provinces.shp'
gdf = gpd.read_file(shp_path)

sample_tif = f'{out_folder}/AQI_cat_target_0.tif'
with rasterio.open(sample_tif) as src:
    raster_crs = src.crs

gdf = gdf.to_crs(raster_crs)

def normalize_name(name):
    s = ''.join(w.capitalize() for w in name.split())
    s = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in s if not unicodedata.combining(c))

gdf['code'] = gdf['NAME_1'].map(normalize_name)

palette = {
    0: (  0, 228,   0),
    1: (255, 255,   0),
    2: (255, 126,   0),
    3: (255,   0,   0),
    4: (143,  63, 151),
    5: (126,   0,  35),
}

colors = [tuple(c/255 for c in palette[k]) for k in sorted(palette)]
cmap   = ListedColormap(colors)
norm   = BoundaryNorm(boundaries=list(range(7)), ncolors=cmap.N)

cmap.set_bad(color='white')  

n_maps = 10
ncols  = 2
nrows  = math.ceil(n_maps/ncols)

fig, axes = plt.subplots(nrows, ncols, figsize=(12, nrows*3),
                         constrained_layout=True)
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i >= n_maps:
        ax.axis('off')
        continue

    path = f'{out_folder}/AQI_cat_target_{i}.tif'
    with rasterio.open(path) as src:
        data  = src.read(1)
        bounds = src.bounds

    plot_data = np.ma.masked_equal(data, 255)

    im = ax.imshow(
        plot_data,
        extent=[bounds.left, bounds.right, bounds.bottom, bounds.top],
        origin='upper',
        cmap=cmap,
        norm=norm
    )

    gdf.boundary.plot(ax=ax, edgecolor='black', linewidth=0.75)
    for _, row in gdf.iterrows():
        c = row.geometry.centroid
        ax.text(
            c.x, c.y, row['code'],
            ha='center', va='center', fontsize=5, color='black',
            path_effects=[pe.withStroke(linewidth=0.5, foreground='black')]
        )

    ax.set_title(f'Target_{i}')
    ax.axis('off')

cbar = fig.colorbar(im, ax=axes.tolist(),
                    orientation='vertical', fraction=0.02, pad=0.01,
                    ticks=range(6))
cbar.set_label('AQI category')
cbar.set_ticklabels([str(i) for i in range(6)])

plt.show()

In [ ]:
tif_path = f'{out_folder}/AQI_cat_target_0.tif'
shp_path = '../Inputs/vn_north_shp/north_provinces.shp'

da = rioxarray.open_rasterio(tif_path, masked=True).squeeze('band', drop=True)
transform = da.rio.transform()
crs       = da.rio.crs
raster_vals = da.values
xmin, xmax = da.x.values.min(), da.x.values.max()
ymin, ymax = da.y.values.min(), da.y.values.max()

gdf = gpd.read_file(shp_path).to_crs(crs)

def normalize_name(name):
    s = ''.join(w.capitalize() for w in name.split())
    s = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in s if not unicodedata.combining(c))

gdf['code'] = gdf['NAME_1'].map(normalize_name)

province_ids = {n: i+1 for i,n in enumerate(gdf['NAME_1'])}
shapes = [(geom, province_ids[n]) for geom,n in zip(gdf.geometry, gdf['NAME_1'])]
province_arr = rasterize(
    shapes,
    out_shape=da.shape,
    transform=transform,
    fill=0,
    dtype='int32'
)

colors = [tuple(c/255 for c in palette[i]) for i in range(6)]
cmap = ListedColormap(colors)
norm = BoundaryNorm(boundaries=list(range(7)), ncolors=cmap.N)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 16), constrained_layout=True)

gdf.plot(ax=ax1, facecolor='none', edgecolor='black', linewidth=0.8)
for _, row in gdf.iterrows():
    c = row.geometry.centroid
    ax1.text(
        c.x, c.y, row['code'],
        ha='center', va='center',
        fontsize=8, color='black',
        path_effects=[pe.withStroke(linewidth=1, foreground='white')]
    )
ax1.set_title("25 tỉnh miền Bắc")
ax1.axis('off')

im = ax2.imshow(
    raster_vals,
    extent=[xmin, xmax, ymin, ymax],
    origin='upper',
    cmap=cmap,
    norm=norm
)
gdf.boundary.plot(ax=ax2, edgecolor='black', linewidth=0.75)
for _, row in gdf.iterrows():
    c = row.geometry.centroid
    ax2.text(
        c.x, c.y, row['code'],
        ha='center', va='center',
        fontsize=8, color='black',
        path_effects=[pe.withStroke(linewidth=1, foreground='black')]
    )
ax2.set_title("AQI Category Map")
ax2.axis('off')

cbar = fig.colorbar(im, ax=ax2, fraction=0.046, pad=0.04, ticks=range(6))
cbar.set_label('AQI category')
cbar.set_ticklabels([str(i) for i in range(6)])

plt.show()